In [1]:
import pandas as pd
# If put the data(.csv) under the same folder, you could use
df = pd.read_csv('garments_worker_productivity.csv')

In [2]:
columns=list(df.columns[1:])
df=df[columns]
df['day']=df['day'].transform(lambda x: x.strip())
df['department']=df['department'].transform(lambda x: 'sewing' if x=='sweing' else x.strip())
df['satisfied']= df['actual_productivity']>=df['targeted_productivity']
df=df.drop(["actual_productivity", "targeted_productivity"], axis=1)
df['wip']=df['wip'].fillna(0)

In [4]:
df['day']=df['day'].transform(lambda x: 0 if x == 'Thursday' else 1 if x == 'Saturday' else 2 if x == 'Sunday' else 3 if x == 'Monday' else 4 if x == 'Tuesday' else 5)
df['quarter']=df['quarter'].transform(lambda x: 0 if x=='Quarter1' else 1 if x=='Quarter2' else 2 if x=='Quarter3' else 3 if x=='Quarter4' else 4)
df['department']=df['department'].transform(lambda x: 0 if x=='sewing' else 1)
poss=list(df['team'].unique())
df['team']=df['team'].transform(lambda x: poss.index(x))
train_df = df.iloc[:int(len(df) * 0.8)]
test_df  = df.iloc[int(len(df) * 0.8):] 
X=train_df[['quarter','department','day','team','satisfied']]
P_s1=X[X['satisfied']==True].shape[0]/(X['satisfied'].shape[0])
P_s0=X[X['satisfied']==False].shape[0]/(X['satisfied'].shape[0])

In [15]:
X= train_df[list(train_df.columns)[4:]]
X

,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,satisfied
0,26.16,1108.0,7080,98,0.0,0,0,59.0,True
1,3.94,0.0,960,0,0.0,0,0,8.0,True
2,11.41,968.0,3660,50,0.0,0,0,30.5,True
3,11.41,968.0,3660,50,0.0,0,0,30.5,True
4,25.90,1170.0,1920,50,0.0,0,0,56.0,True
...,...,...,...,...,...,...,...,...,...
952,29.40,916.0,6960,56,0.0,0,2,58.0,True
953,21.82,1591.0,3240,0,0.0,0,1,52.0,True
954,30.33,398.0,6960,0,0.0,0,1,58.0,False
955,4.60,0.0,3780,0,0.0,0,0,9.0,False


In [13]:
temp= X.iloc[:,:-1]
temp

,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
0,26.16,1108.0,7080,98,0.0,0,0,59.0
1,3.94,0.0,960,0,0.0,0,0,8.0
2,11.41,968.0,3660,50,0.0,0,0,30.5
3,11.41,968.0,3660,50,0.0,0,0,30.5
4,25.90,1170.0,1920,50,0.0,0,0,56.0
...,...,...,...,...,...,...,...,...
952,29.40,916.0,6960,56,0.0,0,2,58.0
953,21.82,1591.0,3240,0,0.0,0,1,52.0
954,30.33,398.0,6960,0,0.0,0,1,58.0
955,4.60,0.0,3780,0,0.0,0,0,9.0


In [12]:
X['satisfied'].unique()

array([ True, False])

In [10]:
import numpy as np

In [ ]:
class MyGaussianNB:
    def fit(self, X, y):
        """
        This method "trains" the classifier. For Gaussian NB, this means calculating
        the mean, standard deviation, and prior probability for each class.
        """
        # Store the names of the features for later use in predict.
        self.X_columns_ = X.columns
        # Get the unique class labels.
        self.classes_ = np.unique(y)
        # Dictionaries to store the calculated parameters.
        self.parameters_ = {}
        self.class_priors_ = {}

        # Iterate over each unique class.
        for i, c in enumerate(self.classes_):
            # Create a subset of the data containing only samples of the current class.
            X_c = X[y == c]

            # For each class, store its mean vector, standard deviation vector, and prior probability.
            self.parameters_[c] = {
                "mean": X_c.mean(axis=0), # Calculate mean for each feature
                "std": X_c.std(axis=0),   # Calculate std for each feature
            }
            # The prior probability is simply the frequency of the class in the dataset.
            self.class_priors_[c] = len(X_c) / len(X)

    def _gaussian_pdf(self, x, mean, std):
        """
        Calculates the probability density of a value 'x' given a Gaussian distribution
        defined by its mean and standard deviation. This is our likelihood function.
        """
        # Add a small epsilon value to the standard deviation to prevent division by zero
        # in case a feature has zero variance for a class.
        epsilon = 1e-9
        numerator = np.exp(-((x - mean)**2) / (2 * (std**2 + epsilon)))
        denominator = np.sqrt(2 * np.pi * (std**2 + epsilon))
        return numerator

    def predict(self, X):
        """
        Makes predictions for a set of new data points X.
        """
        # Convert input to a DataFrame to ensure we can iterate with feature names.
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X, columns=self.X_columns_)

        predictions = []
        # Iterate over each row (each data point) in the test set.
        for index, row in X.iterrows():
            posteriors = {}
            # For each data point, calculate its posterior probability for every possible class.
            for c in self.classes_:
                # We use logarithms to prevent numerical underflow.
                # Multiplying many small probabilities can result in a number too small to store.
                # log(a * b) = log(a) + log(b), which is more stable.
                prior = np.log(self.class_priors_[c])

                # This will be the sum of the log-likelihoods for each feature.
                likelihood = 0
                for feature, value in row.items():
                    # Get the stored mean and std for this feature and class.
                    mean = self.parameters_[c]["mean"][feature]
                    std = self.parameters_[c]["std"][feature]
                    # Calculate the PDF value (the likelihood).
                    pdf_val = self._gaussian_pdf(value, mean, std)
                    # Add the log of the likelihood to the total.
                    # Add a small epsilon to pdf_val to avoid log(0).
                    likelihood += np.log(pdf_val + 1e-9)

                # The posterior is the sum of the log-prior and log-likelihoods.
                posteriors[c] = prior + likelihood

            # The predicted class is the one with the highest log-posterior probability.
            predictions.append(max(posteriors, key=posteriors.get))

        return np.array(predictions)

In [ ]:
my_dict={}
for i in train_df['satisfied'].unique():
    my_dict[i]= {'mean':{i: temp[i].mean() for i in temp.columns}, 'std':{i: temp[i].mean() for i in temp.columns}}
my_dict

{np.True_: {'mean': {'smv': np.float64(15.25889237199582),
   'wip': np.float64(732.7523510971787),
   'over_time': np.float64(4751.755485893417),
   'incentive': np.float64(26.734587251828632),
   'idle_time': np.float64(0.8944618599791013),
   'idle_men': np.float64(0.35214211076280044),
   'no_of_style_change': np.float64(0.13479623824451412),
   'no_of_workers': np.float64(35.1128526645768)},
  'std': {'smv': np.float64(15.25889237199582),
   'wip': np.float64(732.7523510971787),
   'over_time': np.float64(4751.755485893417),
   'incentive': np.float64(26.734587251828632),
   'idle_time': np.float64(0.8944618599791013),
   'idle_men': np.float64(0.35214211076280044),
   'no_of_style_change': np.float64(0.13479623824451412),
   'no_of_workers': np.float64(35.1128526645768)}},
 np.False_: {'mean': {'smv': np.float64(15.25889237199582),
   'wip': np.float64(732.7523510971787),
   'over_time': np.float64(4751.755485893417),
   'incentive': np.float64(26.734587251828632),
   'idle_time'

In [18]:
prior={}
for i in train_df['satisfied'].unique():
    prior[i]= X[X['satisfied']==i].shape[0]/X.shape[0]
prior

{np.True_: 0.7314524555903866, np.False_: 0.26854754440961337}

In [20]:
X= test_df[list(test_df.columns)[4:]]
temp2= X.iloc[:,:-1]
temp2

,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
957,30.10,437.0,7080,32,0.0,0,2,59.0
958,27.48,413.0,6840,38,0.0,0,1,57.0
959,2.90,0.0,3360,0,0.0,0,0,8.0
960,2.90,0.0,960,0,0.0,0,0,8.0
961,26.66,1164.0,6600,23,0.0,0,2,55.0
...,...,...,...,...,...,...,...,...
1192,2.90,0.0,960,0,0.0,0,0,8.0
1193,3.90,0.0,960,0,0.0,0,0,8.0
1194,3.90,0.0,960,0,0.0,0,0,8.0
1195,2.90,0.0,1800,0,0.0,0,0,15.0


In [23]:

predictions = []
for i in range(test_df.shape[0]):
    posteriors={}
    for j in prior.keys():
        post= np.log(prior[j])
        likelihood=0
        for k in temp2.iloc[i].items():
            mean= prior[j]['mean'][k]
            std= prior[j]['std'][k]
            epsilon = 1e-9
            val=np.exp(-((x - mean)**2) / (2 * (std**2 + epsilon)))
            likelihood += np.log(val + 1e-9)
        posteriors[j] = prior + likelihood
    predictions.append(max(posteriors, key=posteriors.get))
np.array(predictions)

TypeError: 'float' object is not subscriptable